In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.4.0
None
False


In [3]:
import torch_scatter
print(torch_scatter.__version__)

2.1.2+pt24cpu


In [3]:
from Bio.SeqUtils import seq1

seq1("ALA")


'A'

In [3]:
import sys
import os
# In thư mục hiện tại và sys.path để kiểm tra
print("Thư mục hiện tại:", os.getcwd())
print("sys.path:", sys.path)
# Thử import module
try:
    from src.utils.molecule_builder import build_molecule
    print("Import thành công!")
except ModuleNotFoundError as e:
    print("Lỗi import:", e)

Thư mục hiện tại: d:\Học tập\Lab314\KhoaLuan\MODELS\Models\ViSDIFF
sys.path: ['c:\\Users\\admin\\anaconda3\\envs\\lab314\\python310.zip', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\DLLs', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\lib', 'c:\\Users\\admin\\anaconda3\\envs\\lab314', '', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\lib\\site-packages', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\lib\\site-packages\\win32', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\lib\\site-packages\\win32\\lib', 'c:\\Users\\admin\\anaconda3\\envs\\lab314\\lib\\site-packages\\Pythonwin']
Lỗi import: No module named 'utils'


In [3]:
from src.utils.molecule_builder import bond_dict
bond_dict
from src.utils.molecule_builder import build_molecule
build_molecule


<function src.utils.molecule_builder.build_molecule(positions, atom_types, dataset_info, add_coords=False, use_openbabel=True)>

In [3]:
import pickle
with open("data/crossdocked_pocket10/index.pkl", "rb") as f:
    index_data = pickle.load(f)
    print(f"Number of entries: {len(index_data)}")
    print(f"First few entries: {index_data[:5]}")
    print(f"Structure of 5 entry: {index_data[0:4]}")

Number of entries: 184057
First few entries: [('1B57_HUMAN_25_300_0/5u98_D_rec_5u98_1kx_lig_tt_min_0_pocket10.pdb', '1B57_HUMAN_25_300_0/5u98_D_rec_5u98_1kx_lig_tt_min_0.sdf', '1B57_HUMAN_25_300_0/5u98_D_rec.pdb', 0.367042), ('1B57_HUMAN_25_300_0/3upr_C_rec_5u98_1kx_lig_tt_min_0_pocket10.pdb', '1B57_HUMAN_25_300_0/3upr_C_rec_5u98_1kx_lig_tt_min_0.sdf', '1B57_HUMAN_25_300_0/3upr_C_rec.pdb', 0.402512), ('1B57_HUMAN_25_300_0/3vri_A_rec_3vri_1kx_lig_tt_min_0_pocket10.pdb', '1B57_HUMAN_25_300_0/3vri_A_rec_3vri_1kx_lig_tt_min_0.sdf', '1B57_HUMAN_25_300_0/3vri_A_rec.pdb', 0.384664), ('1B57_HUMAN_25_300_0/3upr_C_rec_3vri_1kx_lig_tt_min_0_pocket10.pdb', '1B57_HUMAN_25_300_0/3upr_C_rec_3vri_1kx_lig_tt_min_0.sdf', '1B57_HUMAN_25_300_0/3upr_C_rec.pdb', 0.360588), ('1B57_HUMAN_25_300_0/3upr_C_rec_5u98_1kx_lig_tt_docked_3_pocket10.pdb', '1B57_HUMAN_25_300_0/3upr_C_rec_5u98_1kx_lig_tt_docked_3.sdf', '1B57_HUMAN_25_300_0/3upr_C_rec.pdb', 0.705207)]
Structure of 5 entry: [('1B57_HUMAN_25_300_0/5u98_D_r

In [ ]:
import numpy as np
import torch
from pathlib import Path
import pickle
import logging
from tqdm import tqdm
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from src.utils.constants import dataset_params
from src.utils.molecule_builder import build_molecule

# Thiết lập logging
logging.basicConfig(filename='check_crossdock_validity.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def check_ligand_and_pocket(pdbfile, sdffile, atom_dict, dist_cutoff=8.0):

    status = {'valid': False, 'reason': '', 'lig_atoms': 0, 'pocket_atoms': 0, 'build_success': False}

    try:
        pdb_struct = PDBParser(QUIET=True).get_structure('', str(pdbfile))
        if not pdb_struct.get_chains():
            status['reason'] = f"No chains in PDB: {pdbfile}"
            return status
    except Exception as e:
        status['reason'] = f"Failed to parse PDB {pdbfile}: {str(e)}"
        return status

    try:
        ligand = Chem.SDMolSupplier(str(sdffile), sanitize=False)[0]
        if ligand is None:
            status['reason'] = f"No valid molecule in SDF: {sdffile}"
            return status
        try:
            Chem.SanitizeMol(ligand)
        except ValueError as e:
            status['reason'] = f"Sanitize failed for SDF {sdffile}: {str(e)}"
            return status
    except Exception as e:
        status['reason'] = f"Cannot read SDF {sdffile}: {str(e)}"
        return status

    lig_atoms = [a.GetSymbol().capitalize() for a in ligand.GetAtoms()
                 if a.GetSymbol().capitalize() in atom_dict]
    lig_coords = np.array([ligand.GetConformer(0).GetAtomPosition(idx)
                           for idx in range(ligand.GetNumAtoms())])
    status['lig_atoms'] = len(lig_atoms)
    if status['lig_atoms'] == 0:
        status['reason'] = f"Empty valid atoms in ligand {sdffile} (all H or invalid atoms)"
        return status

    pocket_residues = []
    try:
        for residue in pdb_struct[0].get_residues():
            res_coords = np.array([a.get_coord() for a in residue.get_atoms()])
            if len(res_coords) == 0 or len(lig_coords) == 0:
                continue
            min_dist = np.min(np.sqrt(np.sum((res_coords[:, None, :] - lig_coords[None, :, :])**2, axis=-1)))
            if is_aa(residue.get_resname(), standard=True) and min_dist < dist_cutoff:
                pocket_residues.append(residue)
    except Exception as e:
        status['reason'] = f"Error computing pocket residues for {pdbfile}: {str(e)}"
        return status

    if not pocket_residues:
        status['reason'] = f"No pocket residues within {dist_cutoff}A of ligand in {pdbfile}"
        return status

    try:
        full_atoms = np.concatenate(
            [np.array([atom.element.capitalize() for atom in res.get_atoms()])
             for res in pocket_residues], axis=0)
        full_coords = np.concatenate(
            [np.array([atom.coord for atom in res.get_atoms()])
             for res in pocket_residues], axis=0)
        status['pocket_atoms'] = len(full_atoms)
        if status['pocket_atoms'] == 0:
            status['reason'] = f"Empty pocket atoms in {pdbfile}"
            return status
    except Exception as e:
        status['reason'] = f"Error processing pocket atoms for {pdbfile}: {str(e)}"
        return status

    try:
        atom_types = np.array([atom_dict[atm] for atm in lig_atoms])
        lig_coords_torch = torch.tensor(lig_coords, dtype=torch.float32)
        atom_types_torch = torch.tensor(atom_types, dtype=torch.int64)
        mol = build_molecule(lig_coords_torch, atom_types_torch, dataset_params['crossdock_full'], add_coords=True, use_openbabel=False)
        if mol is None:
            status['reason'] = f"Build molecule failed for {sdffile}"
            return status
        Chem.SanitizeMol(mol)
        status['build_success'] = True
    except Exception as e:
        status['reason'] = f"Build/Sanitize molecule failed for {sdffile}: {str(e)}"
        return status

    status['valid'] = True
    status['reason'] = "All checks passed"
    return status

def check_crossdock_database(index_path, data_dir, atom_dict, dist_cutoff=8.0):
    if not index_path.exists():
        print(f"File {index_path} không tồn tại!")
        logging.error(f"File {index_path} không tồn tại!")
        return

    try:
        with open(index_path, 'rb') as f:
            index_data = pickle.load(f)
        print(f"Đã load {len(index_data)} entries từ {index_path}")
        logging.info(f"Đã load {len(index_data)} entries từ {index_path}")

        invalid_samples = []
        empty_ligand_samples = []
        empty_pocket_samples = []
        valid_count = 0
        invalid_count = 0
        empty_lig_count = 0
        empty_pocket_count = 0

        # Thanh tiến trình
        pbar = tqdm(index_data, total=len(index_data), desc="Processing samples")

        for i, entry in enumerate(pbar):
            if not isinstance(entry, (list, tuple)) or len(entry) < 2:
                logging.warning(f"Invalid entry at index {i}: {entry}")
                invalid_samples.append((i, f"Invalid entry: {entry}"))
                invalid_count += 1
                pbar.set_postfix(valid=valid_count, invalid=invalid_count,
                                 empty_lig=empty_lig_count, empty_pocket=empty_pocket_count)
                continue

            pocket_fn, ligand_fn = entry[0], entry[1]
            pdbfile = data_dir / f'{pocket_fn}'
            sdffile = data_dir / f'{ligand_fn}'
            sample_id = f"{pocket_fn}_{ligand_fn}"

            status = check_ligand_and_pocket(pdbfile, sdffile, atom_dict, dist_cutoff)

            if status['valid']:
                valid_count += 1
                logging.info(f"Sample {i} ({sample_id}): VALID - lig={status['lig_atoms']}, pocket={status['pocket_atoms']}, build={status['build_success']}")
            else:
                invalid_samples.append((i, sample_id, status['reason']))
                logging.warning(f"Sample {i} ({sample_id}): INVALID - {status['reason']}")
                invalid_count += 1
                if status['lig_atoms'] == 0:
                    empty_ligand_samples.append((i, sample_id))
                    empty_lig_count += 1
                if status['pocket_atoms'] == 0:
                    empty_pocket_samples.append((i, sample_id))
                    empty_pocket_count += 1

            # Cập nhật thanh tiến trình
            pbar.set_postfix(valid=valid_count, invalid=invalid_count,
                             empty_lig=empty_lig_count, empty_pocket=empty_pocket_count)

        # Báo cáo
        total = len(index_data)
        print(f"\nTổng samples: {total}")
        print(f"Valid: {valid_count} ({valid_count/total*100:.1f}%)")
        print(f"Invalid: {invalid_count} ({invalid_count/total*100:.1f}%)")
        print(f"Empty ligand samples: {len(empty_ligand_samples)}")
        print(f"Empty pocket samples: {len(empty_pocket_samples)}")
        if empty_ligand_samples:
            print("\nEmpty ligand samples (index, name):")
            for inv in empty_ligand_samples[:10]:
                print(f"  - Index {inv[0]} ({inv[1]})")
            if len(empty_ligand_samples) > 10:
                print(f"  ... và {len(empty_ligand_samples)-10} samples khác")
        if empty_pocket_samples:
            print("\nEmpty pocket samples (index, name):")
            for inv in empty_pocket_samples[:10]:
                print(f"  - Index {inv[0]} ({inv[1]})")
            if len(empty_pocket_samples) > 10:
                print(f"  ... và {len(empty_pocket_samples)-10} samples khác")
        if invalid_samples:
            print("\nInvalid samples (index, name, reason):")
            for inv in invalid_samples[:10]:
                print(f"  - Index {inv[0]} ({inv[1]}): {inv[2]}")
            if len(invalid_samples) > 10:
                print(f"  ... và {len(invalid_samples)-10} samples khác")

        logging.info(f"Kiểm tra xong: {valid_count} valid, {invalid_count} invalid, "
                     f"{len(empty_ligand_samples)} empty ligand, {len(empty_pocket_samples)} empty pocket")

    except Exception as e:
        print(f"Lỗi khi xử lý {index_path}: {str(e)}")
        logging.error(f"Lỗi khi xử lý {index_path}: {str(e)}")

data_dir = Path('crossdocked_pocket10')  
index_path = data_dir / 'index.pkl'
atom_dict = dataset_params['crossdock_full']['atom_encoder']
dist_cutoff = 8.0

print("Check CrossDock database...")
check_crossdock_database(index_path, data_dir, atom_dict, dist_cutoff)

print("\n'check_crossdock_validity.log'")

Bắt đầu kiểm tra tính hợp lệ của CrossDock database...
Đã load 184057 entries từ data\crossdocked_pocket10\index.pkl

Tổng samples: 184057
Valid: 0 (0.0%)
Invalid: 184057 (100.0%)
Empty ligand samples: 17587
Empty pocket samples: 17674

Empty ligand samples (index, name):
  - Index 264 (AAKB1_HUMAN_70_159_0/5ufu_B_rec_5ufu_85v_lig_tt_min_0_pocket10.pdb_AAKB1_HUMAN_70_159_0/5ufu_B_rec_5ufu_85v_lig_tt_min_0.sdf)
  - Index 268 (AAKB1_HUMAN_70_159_0/5ezv_B_rec_5ufu_85v_lig_tt_min_0_pocket10.pdb_AAKB1_HUMAN_70_159_0/5ezv_B_rec_5ufu_85v_lig_tt_min_0.sdf)
  - Index 269 (AAKB1_HUMAN_70_159_0/5ezv_B_rec_4cfe_992_lig_tt_docked_0_pocket10.pdb_AAKB1_HUMAN_70_159_0/5ezv_B_rec_4cfe_992_lig_tt_docked_0.sdf)
  - Index 270 (AAKB1_HUMAN_70_159_0/5ufu_B_rec_5ufu_85v_lig_tt_docked_0_pocket10.pdb_AAKB1_HUMAN_70_159_0/5ufu_B_rec_5ufu_85v_lig_tt_docked_0.sdf)
  - Index 272 (AAKB1_HUMAN_70_159_0/5ezv_B_rec_5ufu_85v_lig_tt_docked_1_pocket10.pdb_AAKB1_HUMAN_70_159_0/5ezv_B_rec_5ufu_85v_lig_tt_docked_1.sdf)
  - 